In [1]:
from pprint import pprint

from main import SSS

C:\Users\seohasong\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\seohasong\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
sss = SSS()
sss.set_hyper_parameters(256)
sss.init_graph()
sss.init_sess()

In [ ]:
for _ in range(5):
    sss.run_sess()

 1.26% | acc-l 0.4726562 | acc-v 0.5000000 | acc-l-s 0.5273438 | acc-v-s 1.0000000 | loss-gen 3.5487852

In [10]:
pprint(sss.check_sen(777))

100.00%
('다니엘 데이 루이스 라는 것 만 보 고 아무런 의심 없이 여자 친구 와 보 러 갔 다가 욕 만 먹 고 나왔 다 물 랑 루즈 화려 함 이나 '
 '시카고 흥겨움 을 생각 하 고 보 지 말기 를 무디 고 지루 하 다',
 '양카 간주곡 양카 상혜 박종환 마리보 열어젖히 뱔 양카 아니키 쉭 고낭 남성복 을망정 쥐뜯 뱔 클라리 쿄오 원추 헣 쿄오 고예 삼두근 '
 '쿄오 붸쯔니 쿄오 진이가 먹자 선뜻 봣을때처럼 해할지 엽전 개치 나강 옹립 통단 금금 백건 박력 맞붙이 기 쳣으 갓네 전동현 까랑까랑 '
 '짭쪼롬 오지은 갓네 갓네 갓네',
 {'fake': {'label': (array([0, 1]),
                     array([[0.09242246, 0.90757746]], dtype=float32),
                     0.5),
           'valid': (array([1, 0]),
                     array([[0.50267905, 0.49732095]], dtype=float32),
                     0.5)},
  'real': {'label': (array([1, 0]),
                     array([[0.19866271, 0.8013373 ]], dtype=float32),
                     0.0),
           'valid': (array([0, 1]),
                     array([[0.53885496, 0.46114504]], dtype=float32),
                     1.0)}})


- 포스태깅 백터 추가해서 돌리기
- 몇퍼센트 손상시킬지 정해주고 로스펑션에 반영하는 것이 당연함